In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说吧
### 测试看下来，单纯得跑模型似乎会应该Y值的方差而产生巨大问题，我们需要重新做一次分类，明天的核心任务吧

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
from param_config import config

In [28]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']


###清理异常Y值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]
dfTrain = dfTrain.reset_index(drop=True)

#dfTrain = dfTrain.loc[dfTrain['Y']<=dfTrain['Y'].quantile(0.9)]

In [29]:
def FeatAll(train,pred,pcentByother=False,rankMinus=False,missing=False):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days
    
    ###尝试处理异常值
    '''for var in predictors:
        dfAll.loc[dfAll[var]>=dfAll[var].quantile(0.99)*3,var] = dfAll[var].quantile(0.99)*3'''
        #dfAll.loc[dfAll[var]>=df[var].quantile(0.99)*1.5,var] = np.nan

    ###这个部分将每一列分箱，并且计算基于该列分类的其他列的百分比顺序
    if pcentByother:
        varDict = {}
        bin_set = 15
        for colBase in predictors+['sex','age']:
            if float(dfAll[colBase].isnull().sum())/dfAll.shape[0]>=0.4:
                print('Too many missing in variable %s'%colBase)
                continue
            if dfAll[colBase].nunique()<=5:
                varDict[colBase] = [x for x in predictors if x!=colBase]
            else:
                dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=bin_set,labels=[colBase+'_bin_%d'%i for i in range(bin_set)])
                dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(str)
                dfAll[colBase+'_based'].fillna(colBase+'_bin_NA',inplace = True)
                varDict[colBase+'_based'] = [x for x in predictors if x!=colBase]
        try: 
            dfAllPcentByOther = pd.read_csv('../../Cache/pcent_by_other_bin%d.csv'%bin_set)
        except:
            print('There is no Pcent by other col, will generate and save')
            dfAllPcentByOther = fe.pcent_by_other_col(dfAll,varDict,['ID'])
            dfAllPcentByOther.to_csv('../../Cache/pcent_by_other_bin%d.csv'%bin_set,index=False)
        for var in dfAll.columns:
            if '_based' in var:
                del dfAll[var]
                
    if rankMinus:
        varList = []
        bin_set = 30
        for colBase in predictors:
            dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=bin_set,labels=range(bin_set))
            dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(int)
            varList.append(colBase+'_based')
        try: 
            dfrankMinus = pd.read_csv('../../Cache/rankMinus_bin%d.csv'%bin_set)
        except:
            print('There is no Rank Minus, will generate and save')
            dfrankMinus = fe.var_minus(dfAll,varList,['ID'],0)
            dfrankMinus.to_csv('../../Cache/rankMinus_bin%d.csv'%bin_set,index=False)
            
    ###根据日期放置一个百分比顺序作为尝试
    dfAllPcentByDate = fe.pcent_by_other_col(dfAll,{'date':predictors},['ID']) 
    
    '''dfAll['date'] = pd.qcut(dfAll['date'],q=10,labels=['date_bin_%d'%i for i in range(10)])
    dfAll = pd.concat([dfAll,pd.get_dummies(dfAll['date'])],axis=1)'''
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in
    
    for plus in [['PartII_1','PartII_2'],['PartIII_1','PartIII_2','PartIII_3']]:
        tmpVar =plus[0]
        tmpValue = dfAll[plus[0]].values
        for var in plus[1:]:
            tmpVar = tmpVar +'_plus_' + var
            tmpValue = tmpValue + dfAll[var].values
        dfAll[tmpVar] = tmpValue
    
    dfAll['PartI_9'] = dfAll['PartI_5']-dfAll['PartI_6']-dfAll['PartI_7']
    dfAll['PartII_5'] = dfAll['PartII_2']-dfAll['PartII_3']-dfAll['PartII_4']
    
    for ratio in [['PartII_1','PartII_2'],['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]
        
    for multiply in [['PartI_1','PartI_5'],['PartI_2','PartI_5'],['PartI_3','PartI_5'],['PartI_4','PartI_5'],['PartVI_1','PartVI_2'],['PartV_2','PartV_5'],['PartV_2','PartV_7'],['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]*dfAll[multiply[1]]
    
    
    ###看看missing 的情况
    '''dfAll['missing_cnt'] = dfAll[check_missing].isnull().sum(axis=1)
    for i in range(len(check_missing)):
        dfAll['missing_part%d'%i] = dfAll[check_missing[i]].isnull().astype(int)'''
    
    
    '''for key,group in {'PartI':['PartI_1','PartI_2','PartI_3','PartI_4'],'PartII':['PartII_1','PartII_3','PartII_4']}.items():
        dfAll['group_'+key+'_std'] = dfAll[group].std(axis=1)
        dfAll['group_'+key+'_mean'] = dfAll[group].mean(axis=1)
        dfAll['group_'+key+'_median'] = dfAll[group].median(axis=1)'''
    
    
    try:
        dfAll = dfAll.merge(dfAllPcentByOther,'inner','ID')
    except:
        print('No Pcent by other col')
        
    try:
        dfAll = dfAll.merge(dfrankMinus,'inner','ID')
    except:
        print('No Rank Minus')
        
    
    dfAll = dfAll.merge(dfAllPcentByDate,'inner','ID')
    
    
    ###填补缺失值
    if missing:
        dfAll =dfAll.fillna(dfAll.median())
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred

In [30]:
dfTrain,dfPred = FeatAll(dfTrain,dfPred,True,True,True)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

Too many missing in variable PartIV_1
Too many missing in variable PartIV_2
Too many missing in variable PartIV_3
Too many missing in variable PartIV_4
Too many missing in variable PartIV_5
There is no Pcent by other col, will generate and save


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


There is no Rank Minus, will generate and save


In [31]:
pd.set_option('display.max_rows',len(predictors))
len(predictors)


2693

In [32]:
n_splits=5
early_stop=50
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,'colsample_bytree':0.8}
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,predictors,n_splits,early_stop=early_stop,params=params)

No metric defined, will use rmse in the model
[0]	train-rmse:5.27689	eval-rmse:5.32373
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.21425	eval-rmse:2.3947
[200]	train-rmse:1.24113	eval-rmse:1.59596
[300]	train-rmse:0.966949	eval-rmse:1.443
[400]	train-rmse:0.864655	eval-rmse:1.41771
[500]	train-rmse:0.806839	eval-rmse:1.41151
[600]	train-rmse:0.763432	eval-rmse:1.40868
[700]	train-rmse:0.728247	eval-rmse:1.40791
Stopping. Best iteration:
[671]	train-rmse:0.737594	eval-rmse:1.40752

Best tree is 672, performance is 0.737594, 1.407517
No metric defined, will use rmse in the model
[0]	train-rmse:5.29687	eval-rmse:5.24364
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.23728	eval-rmse:2.26516
[200]	train-rmse:1.26545	eval-rmse:1.43835
[300]	train-rmse:0.982445	ev

In [33]:
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

In [34]:
tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:100]
imp.sort_values('sum_imp',ascending=False).reset_index()

,index,lk,variable,imp_fold1,imp_fold2,imp_fold3,imp_fold4,imp_fold5,sum_imp
0,0,f0,PartIII_1,0.046813,0.042927,0.046149,0.051340,0.043092,0.230322
1,37,f37,age,0.026538,0.031950,0.030766,0.021259,0.028378,0.138891
2,1,f1,PartIII_2,0.023052,0.022311,0.022033,0.021587,0.019109,0.108092
3,2,f2,PartIII_3,0.015045,0.014636,0.012406,0.015393,0.015574,0.073054
4,3,f3,PartII_1,0.011816,0.011870,0.012703,0.010137,0.013090,0.059616
5,1961,f1961,pcent_age_based_PartII_1,0.007813,0.009996,0.017368,0.006288,0.008026,0.049491
6,1206,f1206,pcent_PartIII_1_based_PartII_3,0.002260,0.010531,0.013200,0.011357,0.010032,0.047380
7,4,f4,PartII_2,0.009621,0.009014,0.010818,0.008541,0.007835,0.045828
8,1954,f1954,pcent_age_based_PartI_2,0.007490,0.008925,0.005558,0.005397,0.010797,0.038166
9,5,f5,PartII_3,0.006586,0.008925,0.005260,0.006617,0.005542,0.032929


In [35]:
test_result,result,imp_selected = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop=early_stop,params=params)

No metric defined, will use rmse in the model
[0]	train-rmse:5.27699	eval-rmse:5.32395
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.22195	eval-rmse:2.39351
[200]	train-rmse:1.25712	eval-rmse:1.59763
[300]	train-rmse:0.991342	eval-rmse:1.43511
[400]	train-rmse:0.896957	eval-rmse:1.40596
[500]	train-rmse:0.843238	eval-rmse:1.3978
[600]	train-rmse:0.803441	eval-rmse:1.39458
Stopping. Best iteration:
[630]	train-rmse:0.792877	eval-rmse:1.39342

Best tree is 631, performance is 0.792877, 1.393423
No metric defined, will use rmse in the model
[0]	train-rmse:5.29698	eval-rmse:5.24338
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.24368	eval-rmse:2.26714
[200]	train-rmse:1.28695	eval-rmse:1.43915
[300]	train-rmse:1.0171	eval-rmse:1.28041
[400]	train-rmse:0.917101	ev

In [36]:
for i in range(1,n_splits+1):
    imp_selected['imp_fold%d'%i] = imp_selected['imp_fold%d'%i]/imp_selected['imp_fold%d'%i].sum()
imp_selected['sum_imp'] = imp_selected[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)
imp_selected.sort_values('sum_imp',ascending=False).reset_index()

,index,lk,variable,imp_fold1,imp_fold2,imp_fold3,imp_fold4,imp_fold5,sum_imp
0,1,f1,age,0.059759,0.069151,0.062082,0.053689,0.056802,0.301482
1,0,f0,PartIII_1,0.048156,0.049886,0.058691,0.059843,0.054793,0.271368
2,6,f6,pcent_PartIII_1_based_PartII_3,0.013414,0.024649,0.026693,0.025032,0.023310,0.113098
3,5,f5,pcent_age_based_PartII_1,0.020724,0.024396,0.028085,0.017405,0.015808,0.106419
4,2,f2,PartIII_2,0.024547,0.017246,0.021129,0.020275,0.020631,0.103827
5,8,f8,pcent_age_based_PartI_2,0.021328,0.021200,0.019737,0.017330,0.022038,0.101633
6,10,f10,pcent_PartV_6_based_PartV_5,0.021663,0.018087,0.018433,0.016613,0.018688,0.093484
7,4,f4,PartII_1,0.019517,0.015984,0.015216,0.016575,0.019291,0.086583
8,11,f11,pcent_PartIII_3_based_PartV_2,0.014956,0.020106,0.015912,0.017179,0.016478,0.084631
9,3,f3,PartIII_3,0.017706,0.014301,0.015999,0.015442,0.015674,0.079122


In [37]:
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
print((result['score']>6.8).sum())
print(result['score'].min())
print(result['score'].max())
print(result['score'].mean())
print(result['score'].median())
print("-------------------------")
print(result.iloc[938,:])

54
4.78612
13.1165
5.647828102111816
5.512599468231201
-------------------------
ID         6705.000000
Score_1      13.235938
Score_2      13.076716
Score_3      14.545656
Score_4      12.728355
Score_5      11.995682
score        13.116470
Name: 938, dtype: float64


In [ ]:
dfTrain['Y'].mean()

In [ ]:
###基础变量
1.83977998784

###增加部分比例数据，已经缺失体检项目情况
1.84295123211  imp来看，missing信息没有意义

###上一版去除missing
1.83839589522



###增加一些比值信息
1.84354856365


###根据日期的百分比，效果更加不好了
1.85046680299  变量选择之后 1.83862284729



###前面所有的特征
1.86013896413   变量选择之后  1.85446185976
更改了一下模型的参数 1.81447588639  变量选择之后  1.80873726211
用中位数填补缺失值 1.81136029536  变量选择之后  1.80584671605
将异常值取空 然后按中位数填补 1.83149451674  变量选择之后  1.82105909448
将异常值cap 然后按中位数填补 1.80727635099  变量选择之后  1.81587223984
根据上一版用均值填补缺失（已有cap）  1.81374808226 ///   1.8175380386


###增加更多特征，主要涉及加减乘除
用中位数填补缺失值 1.81045571166 ///  1.78855151825  实际效果极差
不弥补缺失值  1.80566299608  ///  1.79592827524


####增加各列排行相减的变量，中位数填缺失，

In [19]:
other_note ='_more_var_minus'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)